In [ ]:
#Script para gerar Tabela de Notas Fiscais, Itens de nota e Vendas(OLTP)

import random
import pandas as pd
from datetime import datetime

# Exemplo de DataFrame de produtos e preços unitários
data = {
    "NOME": ["Coca-Cola", "Sadia", "Codil", "Tres Coracoes", "Nestle"],
    "PRECO_UNIT": [10.0, 15.5, 8.0, 12.3, 9.8]
}
df_prod_valores = pd.DataFrame(data)

# Função para gerar data e hora aleatória
def gerar_data_hora(ano_inicio, ano_fim):
    ano = random.randint(ano_inicio, ano_fim)
    mes = random.randint(1, 12)
    if mes in [4, 6, 9, 11]:  # Meses com 30 dias
        dia = random.randint(1, 30)
    elif mes == 2:  # Fevereiro
        dia = random.randint(1, 28)
    else:  # Meses com 31 dias
        dia = random.randint(1, 31)

    hora = random.randint(8, 22)
    minuto = random.randint(0, 59)
    segundo = random.randint(0, 59)

    data_completa = datetime(ano, mes, dia, hora, minuto, segundo)
    return data_completa

# Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens, data_nota):
    # Garantir que a quantidade de itens únicos seja pelo menos 1
    quant_item_unicos = random.randint(1, quant_maxima_itens)  # Gerar quantidade aleatória de itens únicos na nota fiscal
    print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")

    lista_saida = []  # Lista para armazenar os itens da nota fiscal
    lista_NOTAFISCAL = []

    id_forma_pgmt = random.randint(1, 8)
    id_cliente = random.randint(1, 1000)
    id_loja = random.randint(1, 150)
    id_cfop = random.randint(1, 5)
    chaveacesso = ''.join([str(random.randint(0, 9)) for _ in range(15)])
    data_criacao = data_nota.date()
    data_atlz = ""
    versao = 1
    
    # Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        if produto in df_prod_valores["NOME"].values:
            preco_unit = df_prod_valores.loc[df_prod_valores["NOME"] == produto, "PRECO_UNIT"].values[0]
            preco_unit = round(float(preco_unit), 2)  # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)  # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            lista_saida.append((data_nota, numero_nota, produto, quantidade, preco_unit, preco_total_item, 
                                id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, data_atlz, versao))

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[5] for item in lista_saida), 2)
    
    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]
   
    lista_NOTAFISCAL.append((data_nota, id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                             data_atlz, versao,preco_total_nota))
    

    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida, columns=["DATA_NOTAFISCAL", "Numero_Nota", "Produto", "QUANTIDADE", "PRECO_UNIT", "TOTAL_ITEM", 
                              "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP","CHAVE_ACESSO","DATA_CRIACAO",
                              "DATA_ATLZ","VERSAO","TOTAL_NOTA"]
    )

    df_2 = pd.DataFrame(
        lista_NOTAFISCAL, columns=["DATA_NOTAFISCAL", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                                   "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO","TOTAL_NOTA"]
    )

    return df_nota_fiscal, df_2

# Lista de produtos
listaprodutosPROVISORIO = ["Coca-Cola", "Sadia", "Codil", "Tres Coracoes", "Nestle"]

# Quantidade de notas fiscais
numero_inicial_nota = 1000
numero_maximo_nota = 1003
quant_maxima_itens = 10

# Lista para armazenar todas as notas fiscais
df_notas_fiscais = []
df_notas_fiscais_2 = []

# Gerar notas fiscais para o intervalo definido
for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
    data_nota = gerar_data_hora(2024, 2024)
    df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, listaprodutosPROVISORIO, quant_maxima_itens, data_nota)
    df_notas_fiscais.append(df_nota_fiscal)
    df_notas_fiscais_2.append(df_2)


# Concatenar todas as notas fiscais em um único DataFrame
df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)


df_ITEMNOTA = pd.DataFrame({
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
               #'ID_PRODUTO':     df_todas_notas[''],
                'QUANTIDADE':     df_todas_notas['QUANTIDADE'],
                'PRECO_UNIT':     df_todas_notas['PRECO_UNIT'],
                'TOTAL_ITEM':     df_todas_notas['TOTAL_ITEM'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

df_NOTAFISCAL = pd.DataFrame({
                'DATA':           df_todas_notas2['DATA_NOTAFISCAL'],
                'TOTAL_NOTA':     df_todas_notas2['TOTAL_NOTA'],
                'ID_FORMA_PGMT':  df_todas_notas2['ID_FORMA_PGMT'],
                'ID_CLIENTE':     df_todas_notas2['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas2['ID_LOJA'],
                'ID_CFOP':        df_todas_notas2['ID_CFOP'],
                'CHAVE_ACESSO':   df_todas_notas2['CHAVE_ACESSO'],
                'DATA_CRIACAO':   df_todas_notas2['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas2['DATA_ATLZ'],
                'VERSAO':         df_todas_notas2['VERSAO'],
})

df_VENDA = pd.DataFrame({
                'DATA':           df_todas_notas['DATA_NOTAFISCAL'],
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                'ID_CLIENTE':     df_todas_notas['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas['ID_LOJA'],
                'ID_FORMA_PGMT':  df_todas_notas['ID_FORMA_PGMT'],
                'TOTAL_VENDA':     df_todas_notas['TOTAL_NOTA'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

#Exibir todas as notas fiscais
#print("\nTodas as Notas Fiscais:")
#print(df_todas_notas)

#display(df_ITEMNOTA)
display(df_NOTAFISCAL)
display(df_VENDA)